# Initial imports

In [ ]:
# to import files from googledrive
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# check version of python

!python --version

install and import required packages

In [ ]:
!pip install -U scikit-learn==1.2.1

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split

# Data for Classification

Load the data for classification

In [ ]:
path = './'

In [ ]:
data_for_classifier = pd.read_csv(path+'gold_std_clean.csv')
#data_for_classifier = pd.read_csv(path+'gold_std_for_anatomy.csv')

take a look at first 5 rows of the data and drop columns not required, or rename as needed

In [ ]:
data_for_classifier.head()

In [ ]:
data_for_classifier = data_for_classifier.drop(columns=['some cells have vals for 5th round vlookup because docid have altered post medcat', 'pain_character', 'pain_management', 'self_harm'  ])

In [ ]:
data_for_classifier['label'].value_counts()

In [ ]:
#ignore --- convert labels (relevant Yes = 1, No = 0, Negated = 0)

data_for_classifier['relevant'] = np.where(data_for_classifier['relevant'] == "Yes", int(1), data_for_classifier['relevant'])
data_for_classifier['relevant'] = np.where(data_for_classifier['relevant'] == "yes", int(1), data_for_classifier['relevant'])
data_for_classifier['relevant'] = np.where(data_for_classifier['relevant'] == "YES", int(1), data_for_classifier['relevant'])
data_for_classifier['relevant'] = np.where(data_for_classifier['relevant'] == "No", int(0), data_for_classifier['relevant'])
data_for_classifier['relevant'] = np.where(data_for_classifier['relevant'] == "no", int(0), data_for_classifier['relevant'])
data_for_classifier['relevant'] = np.where(data_for_classifier['relevant'] == "Negated", int(0), data_for_classifier['relevant'])
data_for_classifier['relevant'] = np.where(data_for_classifier['relevant'] == "negated", int(0), data_for_classifier['relevant'])


In [ ]:
data_for_classifier.describe()

In [ ]:
#ignore --- rename columns

data_for_classifier.rename(columns={'relevant': 'label'}, inplace=True)


# Split the data into train and test (80:20)

In [ ]:
training_data, testing_data = train_test_split(data_for_classifier, test_size=0.2, random_state=1)

In [ ]:
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

In [ ]:
training_data.head()

In [ ]:
training_data.describe()

In [ ]:
training_data['label'].value_counts()

In [ ]:
testing_data.head()

In [ ]:
testing_data.to_csv('test_data_pain.csv')

In [ ]:
testing_data.describe()

In [ ]:
testing_data['label'].value_counts()

# BERT Classification 1

(Run the second BERT classification below for confidence intervals)

BERT classifier

Install and import pacakges needed

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
!pip install torch

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip install transformers==4.5.0

In [ ]:
!pip install transformers

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import StratifiedKFold
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset
from tqdm import trange
from transformers import BertForSequenceClassification, BertTokenizerFast, BertTokenizer, AdamW, AutoModel, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

In [ ]:
import torch.nn as nn

FOR bert_base_uncased

In [ ]:
BERT_tokenizer = 'bert-base-uncased'
#BERT_tokenizer = 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
MAX_TKN_LEN=511

In [ ]:
# import BERT-base pretrained model
#trained_bert_model = AutoModel.from_pretrained('bert-base-uncased')
#trained_bert_model = AutoModelForSequenceClassification.from_pretrained(BERT_tokenizer, num_labels = 2, output_attentions = False, output_hidden_states = False)
#trained_bert_model = AutoModel.from_pretrained(BERT_tokenizer, num_labels = 2, output_attentions = False, output_hidden_states = False)
trained_bert_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
trained_bert_model.cuda()

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
#tokenizer = AutoTokenizer.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')


FOR SAPBERT

In [ ]:
#BERT_tokenizer = 'bert-base-uncased'
BERT_tokenizer = 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
MAX_TKN_LEN=511

In [ ]:
# import BERT-base pretrained model
#trained_bert_model = AutoModel.from_pretrained('bert-base-uncased')
#trained_bert_model = AutoModelForSequenceClassification.from_pretrained(BERT_tokenizer, num_labels = 2, output_attentions = False, output_hidden_states = False)
#trained_bert_model = AutoModel.from_pretrained(BERT_tokenizer, num_labels = 2, output_attentions = False, output_hidden_states = False)
#trained_bert_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
trained_bert_model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
trained_bert_model.cuda()

# Load the BERT tokenizer
#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')


Training data

In [ ]:
training_data.head()

In [ ]:
#ifnore if you don't have these columns already
training_data = training_data.drop(columns=['task','word_count', 'char_count', 'unique_word_count'])

In [ ]:
training_data.shape

In [ ]:
text_col = 'context'
label_col = 'label'


In [ ]:
training_data['context'] = training_data['context'].apply(lambda x: x.strip().lower())
training_data['context']= training_data['context'].str.replace('[^\w\s]','')
print('num annotations:', len(training_data), '\n\n', training_data['label'].value_counts(), '\n\n', training_data[['label', 'context']].head(5))

# BERT functions

In [ ]:
# Create the DataLoaders for our training and validation sets.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.
def create_dataloader(train_dataset, val_dataset, batch_size=16):
    # We'll take training samples in random order.
    train_dataloader = DataLoader(
        train_dataset,  # The training samples.
        sampler=RandomSampler(train_dataset),  # Select batches randomly
        batch_size=batch_size  # Trains with this batch size.
    )
    # For validation the order doesn't matter, so we'll just read them sequentially.
    validation_dataloader = DataLoader(
        val_dataset,  # The validation samples.
        sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
        batch_size=batch_size  # Evaluate with this batch size.
    )

    return [train_dataloader, validation_dataloader]

In [ ]:
# convert labels to numerical values
def convert_to_cat(labels, binary=False):
    if not isinstance(labels, pd.Series):
        labels = pd.Series(labels)
    if binary:
        main_val = labels.mode()[0]
        other_val = [x for x in labels.unique() if x != main_val]
        labels = np.where(labels != main_val, 1, 0)
        corresp = pd.DataFrame([main_val, 'other'], columns=['old_label'])
    elif np.issubdtype(labels.dtype, np.number) and (labels.min() >= 0):
        print('labels already in a good format')
        return {'labels': labels, 'categories': pd.DataFrame(labels.unique())}
    else:
        cats = labels.astype('category').cat
        labels = cats.codes.astype('long')  # convert annotations to integers
        corresp = pd.DataFrame(cats.categories, columns=['old_label'])
    corresp.index.rename('new_label', inplace=True)
    print('labels have been transformed for the model:\n\n', corresp)
    return {'labels': labels, 'categories': corresp}

In [ ]:
# Function to calculate performance of our predictions vs labels
def perf_metrics(preds, labels, average='weighted', debug=False):
    try:
        pred_flat = np.argmax(preds, axis=1).flatten()
        # pred_flat = torch.max(pred_vec, 1)[1]
    except:
        print('only 1 dimension in labels prediction, no need for argmax')
        pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    acc = accuracy_score(labels_flat, pred_flat)
    f1 = f1_score(labels_flat, pred_flat, average=average)
    p = precision_score(labels_flat, pred_flat, average=average)
    r = recall_score(labels_flat, pred_flat, average=average)
    if debug:
        print("PERF -- Acc: {:.3f} F1: {:.3f} Precision: {:.3f} Recall: {:.3f}".format(acc, f1, p, r))
    return {'f1': f1, 'acc': acc, 'p': p, 'r': r}


def perf_metrics_classes(preds, labels):
    try:
        pred_flat = np.argmax(preds, axis=1).flatten()
    except:
        print('only 1 dimension in labels prediction, no need for argmax')
        pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    report = classification_report(labels_flat, pred_flat, output_dict=True)
    df = pd.DataFrame(report).sort_index().transpose()
    # df['accuracy'] = accuracy_score(labels, preds)

    return df

In [ ]:
# functions to handle different devices
def to_cpu(vec, detach=True):
    try:
        vec = vec.detach().cpu().numpy() if detach else vec.to('cpu').numpy()
    except:
        vec = vec.detach().numpy() if detach else vec.numpy()
    return vec


def model_to_cpu(model):
    try:
        model.to("cpu")
    except:
        pass


def batch_to_gpu(batch, device=None):
    if device is None:
        return batch
    try:  # we're using a GPU
        batch = tuple(t.to(device) for t in batch)
    except:
        batch = tuple(t for t in batch)
    return batch

def get_device():
    # specify GPU device
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if device != 'cpu':
            print('number GPUs used:', torch.cuda.device_count())
            print('device name:', torch.cuda.get_device_name(0))
    except:
        device = None
        print('no CUDA capable device detected')
    return device

In [ ]:
def prep_BERT_dataset(sentences, labels=None, BERT_tokenizer='cambridgeltl/SapBERT-from-PubMedBERT-fulltext', MAX_TKN_LEN=511, debug=False):
    """
    JC: change tokenizer to bert-base-uncased or 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'

    prepares data for BERT fine-tuning
    :param sentences: list or array-like. list of texts to classify
    :param labels: list or array-like, default None. list of labels corresponding to sentences
    :param BERT_tokenizer: string, default 'bert-base-uncased'. BERT base model used
    :param MAX_TKN_LEN: integer, default 511. see https://github.com/huggingface/transformers/issues/2446
    :param debug: bool, default False. set to True to display inetrmediate results
    :return: prepared dataset (torch Dataset) and nmber of dictinct labels
    """
    sentences = pd.Series(sentences)
    # load relevant data and add special tokens for BERT to work properly
    sentences = ["[CLS] " + query + " [SEP]" for query in sentences]
    if labels is not None:
        labels = convert_to_cat(labels, binary=False)['labels']
    else:
        labels = pd.Series([1] * len(sentences))
    if debug: print(sentences[0])

    # Tokenize with BERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(BERT_tokenizer, do_lower_case=True)
    if MAX_TKN_LEN is not None:
        print('cutting the length of tokens to', MAX_TKN_LEN)
        tokenized_texts = [tokenizer.tokenize(sent)[0:MAX_TKN_LEN] for sent in sentences]
    else:
        tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]
    if debug:
        print("Tokenize the first sentence:")
        print(len(tokenized_texts), len(input_ids), len(input_ids[0]))
        print(tokenized_texts[0], input_ids[0])

    # add paddding to input_ids
    input_ids_padded = pad_sequence([torch.tensor(i) for i in input_ids]).transpose(0, 1)
    if debug: print(input_ids_padded.size(), len(input_ids_padded))

    # Create attention masks
    attention_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids_padded:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    # create dataset
    dataset = TensorDataset(input_ids_padded, torch.tensor(attention_masks), torch.tensor(labels))
    num_labels = labels.nunique()
    return {'dataset': dataset, 'num_labels': num_labels}

In [ ]:
def finetune_BERT_pytorch(model, train_dataloader, validation_dataloader, n_epochs=3, output_dir='./sapbert_CE_may11/'):
    """
    JC: update output_dir or say None

    fine-tunes Bert for text classification
    :param model: base Bert model
    :param train_dataloader: Tensor Dataset. training data
    :param validation_dataloader: Tensor Dataset. testing data
    :param n_epochs: integer, default 5. number of epochs
    :param output_dir: string, default None. directory where to save the fine-tuned model
    :return:    model: fine-tuned Bert model with highest performance over k folds
                stats: high level performance statistics
                stats_classes: detailed performance statistics
    """
    ###################################################################################
    # BERT fine-tuning parameters
    device = get_device()
    try:
        model.cuda()
    except:
        device = None
        print('using CPU, this will be slow!')
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)

    # BERT training loop
    train_loss_set = []
    best_f1, best_epoch = 0, 0
    for _ in trange(n_epochs, desc="Epoch"):
        ###################################################################################
        ## TRAINING

        # Set our model to training mode
        model.train()
        # Tracking variables
        tr_loss, tr_perf, tr_perf_classes = 0, Counter({}), pd.DataFrame()
        running_len = 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs['loss'], outputs['logits']
            train_loss_set.append(loss.item())
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            tmp_tr_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_tr_perf.update((k, v * len(b_input_ids)) for k, v in tmp_tr_perf.items())
            running_len += len(b_input_ids)
            tr_perf = tr_perf + Counter(tmp_tr_perf)
            tmp_tr_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            tr_perf_classes = pd.concat((tr_perf_classes, tmp_tr_perf_classes))

        # print('classes detail \n\n', tr_perf_classes)
        tr_perf = {k: v / running_len for k, v in tr_perf.items()}
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[
            ['f1-score', 'precision', 'recall']].multiply(tr_perf_classes['support'], axis="index")
        tr_perf_classes = tr_perf_classes.groupby(tr_perf_classes.index).sum()
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[['f1-score', 'precision', 'recall']].div(
            tr_perf_classes['support'], axis="index")
        print('TRAIN - Loss: {:.3f} - F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(tr_loss / (1 + step),
                                                                                         tr_perf['f1'], tr_perf['acc'],
                                                                                         tr_perf['p'], tr_perf['r']))

        ###################################################################################
        ## VALIDATION

        # Put model in evaluation mode
        model.eval()
        # Tracking variables
        eval_perf, eval_perf_classes = Counter({}), pd.DataFrame()
        running_len = 0
        # Evaluate data for one epoch
        for step, batch in enumerate(validation_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():  # Telling the model not to compute or store gradients, saving memory and speeding up validation
                # Forward pass, calculate logit predictions
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
                loss, logits = outputs['loss'], outputs['logits']
            # Update tracking variables
            tmp_eval_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_eval_perf.update((k, v * len(b_input_ids)) for k, v in tmp_eval_perf.items())
            # print('STEP:', step, 'LEN', len(b_input_ids), tmp_eval_perf)
            running_len += len(b_input_ids)
            eval_perf = eval_perf + Counter(tmp_eval_perf)
            tmp_eval_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            eval_perf_classes = pd.concat((eval_perf_classes, tmp_eval_perf_classes))

        eval_perf = {k: v / running_len for k, v in
                     eval_perf.items()}  # eval_perf = {k:v/(1+step) for k,v in eval_perf.items()}
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[
            ['f1-score', 'precision', 'recall']].multiply(eval_perf_classes['support'], axis="index")
        eval_perf_classes = eval_perf_classes.groupby(eval_perf_classes.index).sum()
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[
            ['f1-score', 'precision', 'recall']].div(eval_perf_classes['support'], axis="index")
        print('TEST -- F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(eval_perf['f1'], eval_perf['acc'],
                                                                          eval_perf['p'], eval_perf['r']))

        # store perf metrics and model
        if eval_perf['f1'] >= best_f1:
            best_f1 = eval_perf['f1']
            best_epoch = _ + 1
            stats_to_save = eval_perf
            tr_perf_classes['dataset'] = 'train'
            eval_perf_classes['dataset'] = 'test'
            stats_classes_to_save = pd.concat([tr_perf_classes, eval_perf_classes])

        if output_dir is not None: #save model with best F1
            try:
                print('saving...')
                model.save_pretrained(output_dir)
                stats_classes_to_save.to_csv(output_dir + '/stats.csv', header=True)
            except:
                print('model not saved, please enter valid path')
        print('best F1 score obtained: {:.3f} at epoch {}'.format(best_f1, best_epoch))

    return {'stats': stats_to_save, 'stats_classes': stats_classes_to_save, 'model': model}

In [ ]:
def run_BERT(model, train_dataloader, validation_dataloader, n_epochs=3, output_dir='./sapbert_CE_may11/'):
    """
    JC: update output_dir or say None

    fine-tunes Bert for text classification
    :param model: base Bert model
    :param train_dataloader: Tensor Dataset. training data
    :param validation_dataloader: Tensor Dataset. testing data
    :param n_epochs: integer, default 5. number of epochs
    :param output_dir: string, default None. directory where to save the fine-tuned model
    :return:    model: fine-tuned Bert model with highest performance over k folds
                stats: high level performance statistics
                stats_classes: detailed performance statistics
    """
    ###################################################################################
    # BERT fine-tuning parameters
    device = get_device()
    try:
        model.cuda()
    except:
        #device = None
        print('could not move Bert to Cuda, using CPU, this will be slow!')
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
    loss_fn = nn.CrossEntropyLoss() #JC try this?

    # BERT training loop
    train_loss_set = []
    best_f1, best_epoch = 0, 0
    for _ in trange(n_epochs, desc="Epoch"):
        ###################################################################################
        ## TRAINING

        # Set our model to training mode
        model.train()
        # Tracking variables
        tr_loss, tr_perf, tr_perf_classes = 0, Counter({}), pd.DataFrame()
        running_len = 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            #loss, logits = outputs['loss'], outputs['logits']
            #train_loss_set.append(loss.item())

            #if using cross entropy then comment out loss, logits and use the lines below
            ##loss = output[0] #keep this commented
            logits = outputs[1]
            loss = loss_fn(logits,b_labels)
            print(loss)
            train_loss_set.append(loss.item())


            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            tmp_tr_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_tr_perf.update((k, v * len(b_input_ids)) for k, v in tmp_tr_perf.items())
            running_len += len(b_input_ids)
            tr_perf = tr_perf + Counter(tmp_tr_perf)
            tmp_tr_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            tr_perf_classes = pd.concat((tr_perf_classes, tmp_tr_perf_classes))

        # print('classes detail \n\n', tr_perf_classes)
        tr_perf = {k: v / running_len for k, v in tr_perf.items()}
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[
            ['f1-score', 'precision', 'recall']].multiply(tr_perf_classes['support'], axis="index")
        tr_perf_classes = tr_perf_classes.groupby(tr_perf_classes.index).sum()
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[['f1-score', 'precision', 'recall']].div(
            tr_perf_classes['support'], axis="index")
        print('TRAIN - Loss: {:.3f} - F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(tr_loss / (1 + step),
                                                                                         tr_perf['f1'], tr_perf['acc'],
                                                                                         tr_perf['p'], tr_perf['r']))

        ###################################################################################
        ## VALIDATION

        # Put model in evaluation mode
        model.eval()
        # Tracking variables
        eval_perf, eval_perf_classes = Counter({}), pd.DataFrame()
        running_len = 0
        # Evaluate data for one epoch
        for step, batch in enumerate(validation_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():  # Telling the model not to compute or store gradients, saving memory and speeding up validation
                # Forward pass, calculate logit predictions
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
                loss, logits = outputs['loss'], outputs['logits']
            # Update tracking variables
            tmp_eval_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_eval_perf.update((k, v * len(b_input_ids)) for k, v in tmp_eval_perf.items())
            # print('STEP:', step, 'LEN', len(b_input_ids), tmp_eval_perf)
            running_len += len(b_input_ids)
            eval_perf = eval_perf + Counter(tmp_eval_perf)
            tmp_eval_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            eval_perf_classes = pd.concat((eval_perf_classes, tmp_eval_perf_classes))

        eval_perf = {k: v / running_len for k, v in
                     eval_perf.items()}  # eval_perf = {k:v/(1+step) for k,v in eval_perf.items()}
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[
            ['f1-score', 'precision', 'recall']].multiply(eval_perf_classes['support'], axis="index")
        eval_perf_classes = eval_perf_classes.groupby(eval_perf_classes.index).sum()
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[
            ['f1-score', 'precision', 'recall']].div(eval_perf_classes['support'], axis="index")
        print('TEST -- F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(eval_perf['f1'], eval_perf['acc'],
                                                                          eval_perf['p'], eval_perf['r']))

        # store perf metrics and model
        if eval_perf['f1'] >= best_f1:
            best_f1 = eval_perf['f1']
            best_epoch = _ + 1
            stats_to_save = eval_perf
            tr_perf_classes['dataset'] = 'train'
            eval_perf_classes['dataset'] = 'test'
            stats_classes_to_save = pd.concat([tr_perf_classes, eval_perf_classes])

        if output_dir is not None: #save model with best F1
            try:
                print('saving...')
                model.save_pretrained(output_dir)
                stats_classes_to_save.to_csv(output_dir + '/stats.csv', header=True)
            except:
                print('model not saved, please enter valid path')
        print('best F1 score obtained: {:.3f} at epoch {}'.format(best_f1, best_epoch))

    return {'stats': stats_to_save, 'stats_classes': stats_classes_to_save, 'model': model}

In [ ]:
def train_BERT(sentences, labels, BERT_tokenizer='cambridgeltl/SapBERT-from-PubMedBERT-fulltext', test_size=0.1,
                        n_epochs=3, batch_size=16, output_dir='./sapbert_CE_may11/', MAX_TKN_LEN=511):
    """
    JC: update output_dir or say None;
    JC: update tokenizer to bert-base-uncased or 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'

    formats dataset and fine-tunes Bert model on it
    :param sentences: list or array-like. list of texts to classify
    :param labels: list or array-like. list of labels corresponding to sentences
    :param BERT_tokenizer: string, default 'bert-base-uncased'. BERT base model used
    :param test_size: float, default 0.10. train/test split size
    :param n_epochs: integer, default 5. number of epochs used to fine-tune Bert
    :param batch_size: integer, default 32. how many samples pper batch to load
    :param output_dir: string, default None. directory where to save the fine-tuned model
    :param MAX_TKN_LEN: integer, default 511. see https://github.com/huggingface/transformers/issues/2446
    :return:
    """
    print('formating dataset')
    prep_data = prep_BERT_dataset(sentences=sentences, labels=labels, BERT_tokenizer=BERT_tokenizer,
                                  MAX_TKN_LEN=MAX_TKN_LEN)
    dataset = prep_data['dataset']
    num_labels = prep_data['num_labels']
    # split into train/test
    print('splitting in train/test sets')
    test_len = int(len(dataset) * test_size)
    train_len = len(dataset) - test_len
    print('test set:', test_len, 'train set:', train_len)
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, test_len])
    # Create the DataLoaders for our training and validation sets.
    train_dataloader, validation_dataloader = create_dataloader(train_dataset, val_dataset, batch_size=batch_size)
    # Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
    print('loading pre-trained BERT')
    pretrained_model = BertForSequenceClassification.from_pretrained(BERT_tokenizer, num_labels=num_labels, output_attentions = False, output_hidden_states = False)
    # train and evaluate BERT
    print('training BERT')
    res = finetune_BERT_pytorch(pretrained_model, train_dataloader, validation_dataloader, n_epochs=n_epochs, output_dir=output_dir)
    return res

In [ ]:
# load pre-trained model and classify a new sentence
def load_and_run_BERT(sentences, trained_bert_model, BERT_tokenizer='cambridgeltl/SapBERT-from-PubMedBERT-fulltext', MAX_TKN_LEN=511,
                      batch_size=16):
    """
    JC: update tokenizer to bert-base-uncased or 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'

    loads pre-trained Bert model and runs it on new text to classify
    :param sentences: list or array-like. list of texts to classify
    :param trained_bert_model: string or pytorch model. pre-trained model name or path
    :param BERT_tokenizer: string, default 'bert-base-uncased'. BERT base model used
    :param MAX_TKN_LEN: integer, default 511. see https://github.com/huggingface/transformers/issues/2446
    :param batch_size: integer, default 32. how many samples pper batch to load
    :return: dataframe of sentences classified along with probaility scores
    """
    if isinstance(trained_bert_model, str):  # load trained BERT model if needed
        trained_bert_model = BertForSequenceClassification.from_pretrained(trained_bert_model)
    preds_class, probs, preds = [], [], pd.DataFrame()
    sentences = pd.Series(sentences)
    sentences_dataset = \
    prep_BERT_dataset(sentences, labels=None, BERT_tokenizer=BERT_tokenizer, MAX_TKN_LEN=MAX_TKN_LEN)['dataset']
    # b_input_ids, b_input_mask, b_labels = sentences_dataset.tensors
    validation_dataloader = DataLoader(sentences_dataset, sampler=SequentialSampler(sentences_dataset),
                                       batch_size=batch_size)
    for step, batch in enumerate(validation_dataloader):
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = trained_bert_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs['loss'], outputs['logits']
        preds_curr = logits.detach().numpy()
        preds = preds.append(pd.DataFrame(preds_curr), ignore_index=True)
        probs = np.append(probs, np.max(np.exp(preds_curr) / (1 + np.exp(preds_curr)), axis=1))
        preds_class = np.append(preds_class, np.argmax(preds_curr, axis=1).flatten())

    # put results in nice format
    preds['sentences'] = sentences
    preds['preds'] = preds_class
    preds['probs'] = probs
    #preds['label_goldstd'] = df['label'] #testing something - remove this if not needed or causing errors - use only when you want to do error analysis
    return preds


In [ ]:
# TO RUN K-FOLD VALIDATION
def run_KFOLD(dataset, base_model, model_trainer, base_model_loader=None,
             n_splits=10, random_state=42, n_epochs=3, **kwargs):

    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    labels = pd.Series([1] * len(dataset))
    # tracking variables
    best_f1, fold_nb = 0, 0
    stats, stats_classes = pd.DataFrame(), pd.DataFrame()
    # run k fold
    for train_ix, test_ix in kfold.split(labels, labels):
        fold_nb +=1
        # need to load each time otherwise remembers training from previous fold
        if base_model_loader is not None:
            base_model_tmp = base_model_loader(base_model, **kwargs)
        else:
            base_model_tmp = base_model
        print('####################### RUNNING FOLD:', fold_nb)
        train_dataset = torch.utils.data.Subset(dataset, train_ix)
        val_dataset = torch.utils.data.Subset(dataset, test_ix)
        print(type(train_dataset), ' train set:', len(train_dataset), ' test set:',len(val_dataset))
        train_dataloader, validation_dataloader = create_dataloader(train_dataset, val_dataset)
        print('training and evaluating model')
        res = model_trainer(base_model_tmp, train_dataloader, validation_dataloader, n_epochs=n_epochs, output_dir=None)
        del base_model_tmp
        # store perf metrics and model
        stats_tmp = pd.DataFrame.from_dict(res['stats'],orient='index', columns=['value'])
        stats_tmp['fold'] = fold_nb
        stats = pd.concat([stats, stats_tmp])
        res['stats_classes']['fold'] = fold_nb
        stats_classes = pd.concat([stats_classes, res['stats_classes']])
        if res['stats']['f1'] >= best_f1:
            best_f1 = res['stats']['f1']
            res_to_save = res


    print('best F1 score obtained across splits: {:.3f}'.format(best_f1))
    return {'stats':stats, 'stats_classes':stats_classes, 'model':res_to_save['model']}

In [ ]:
def BERT_KFOLD(sentences, labels, BERT_tokenizer='cambridgeltl/SapBERT-from-PubMedBERT-fulltext',
               n_splits=10, random_state=42, n_epochs=3, output_dir='./sapbert_CE_may11/', MAX_TKN_LEN=511):
    """
    JC: update output_dir or say None;
    JC: update tokenizer to bert-base-uncased or 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'

    fine-tunes Bert model using k-old cross validation
    :param sentences: list or array-like. list of texts to classify
    :param labels: list or array-like. list of labels corresponding to sentences
    :param BERT_tokenizer: string, default 'bert-base-uncased'. BERT base model used
    :param n_splits: integer, default None. number of folds to use
    :param random_state: integer, default 42. random seed to initialize folds generation
    :param n_epochs: integer, default 5. number of epochs used to fine-tune Bert
    :param output_dir: string, default None. directory where to save the fine-tuned model
    :param MAX_TKN_LEN: integer, default 511. see https://github.com/huggingface/transformers/issues/2446
    :return:    model: fine-tuned Bert model with highest performance over k folds
                stats: high level performance statistics
                stats_classes: detailed performance statistics
    """
    prep_data = prep_BERT_dataset(sentences=sentences, labels=labels, BERT_tokenizer=BERT_tokenizer,
                                  MAX_TKN_LEN=MAX_TKN_LEN)
    dataset = prep_data['dataset']
    res = run_KFOLD(dataset=dataset, base_model=BERT_tokenizer, model_trainer=run_BERT,
                    base_model_loader=BertForSequenceClassification.from_pretrained,
                    n_splits=n_splits, random_state=random_state, n_epochs=n_epochs, num_labels=prep_data['num_labels'])
    if output_dir is not None:
        try:
            print('saving model in:', output_dir)
            res['model'].save_pretrained(output_dir)
            res['stats_classes'].to_csv(output_dir + '/stats.csv', header=True)
        except:
            print('model not saved, please enter valid path')
    return res

In [ ]:
#Confidence Intervals (Credit to Diana Shamsutdinova)

def f1_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    f1 = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        f1[bs] = (f1_score(tp_bootstrap["t"], tp_bootstrap["p"], average='weighted')) #added , average='weighted'
    res = [f1.mean(), np.quantile(f1,0.025), np.quantile(f1,0.975), f1.std()]
    return res

def precision_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    p = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        p[bs] = (precision_score(tp_bootstrap["t"], tp_bootstrap["p"], average='weighted')) #added , average='weighted'
    res = [p.mean(), np.quantile(p,0.025), np.quantile(p,0.975), p.std()]
    return res

def recall_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (recall_score(tp_bootstrap["t"], tp_bootstrap["p"], average='weighted')) #added , average='weighted'
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

def class_0_f1_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (f1_score(tp_bootstrap["t"], tp_bootstrap["p"], pos_label=0, average='binary'))
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

def class_1_f1_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (f1_score(tp_bootstrap["t"], tp_bootstrap["p"], pos_label=1, average='binary'))
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

In [ ]:
# fine tune Bert - update output_dir or say None
bert_model = train_BERT(sentences=data_for_classifier['context'].to_list(), labels=data_for_classifier['label'].to_list(), test_size=0.2, n_epochs=5, output_dir='./sapbert_CE_may11/')
bert_model['stats']

In [ ]:
bert_model

In [ ]:
# run pre-trained model --- doesn't work - RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument index in method wrapper_index_select)
load_and_run_BERT(sentences=['hello my name is link i am in love with princess zelda', 'this is just a test sentence'],
                  trained_bert_model=bert_model['model'],
                  BERT_tokenizer=BERT_tokenizer)

In [ ]:
# run k-fold validation
kf = BERT_KFOLD(sentences=data_for_classifier['context'].to_list(), labels=data_for_classifier['label'].to_list(), n_splits=10, BERT_tokenizer=BERT_tokenizer, n_epochs=5,
                random_state=666)
print(kf['stats'], '\n\n', kf['stats_classes'])

In [ ]:
kf['stats_classes'].to_csv('./sapbert_CE_may11/kf_stats_classes.csv')

In [ ]:
kf['stats'].to_csv('./sapbert_CE_may11/kf_stats.csv')

confidence intervals - this wont work here because the labels and pred variables are from the second bert method below

In [ ]:
f1_conf_int(flat_true_labels, flat_predictions) #gives mean, lower ci, upper ci, std

In [ ]:
f1_score(flat_true_labels, flat_predictions)

In [ ]:
class_0_f1_conf_int(flat_true_labels, flat_predictions)

In [ ]:
class_1_f1_conf_int(flat_true_labels, flat_predictions)

In [ ]:
print(classification_report(flat_true_labels, flat_predictions))

In [ ]:
bert_report = classification_report(flat_true_labels, flat_predictions, output_dict=True)
df_bert = pd.DataFrame(bert_report).transpose()
df_bert.to_csv('./sapbert_CE_may11/classification_report.csv', index= True)
df_bert.head()

In [ ]:
data_for_classifier.head()

In [ ]:
testing_data.describe()

In [ ]:
testing_data.head()

In [ ]:
# RUN PRETRAINED BERT AND ANALYSE ERRORS
BERT_tokenizer = 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext' #or can be 'bert-base-uncased'
# model_path = data_path+'bert_models/'+dom+'_biobert'
model_path = './sapbert_CE_may11' #update accordingly
#text_col = 'clean_text'
text_col = 'context'

#df = pd.read_excel(data_path+'/cognition_adjudicated_clean.xlsx')
df = pd.read_csv('test_data_pain.csv', usecols=['label', 'context', 'BrcID', 'docid', 'text', 'value'])
                 #columns=['Brcid', 'CN_Doc_ID','doc_date', 'doc_type', 'kw', 'domain', 'start_kw', 'clean_text'])
# df = df.loc[df.domain == dom].reset_index()
df[text_col] = df[text_col].apply(lambda x: str(x).strip().lower())

res= load_and_run_BERT(sentences=df[text_col]
                       , trained_bert_model=model_path
                       , BERT_tokenizer=BERT_tokenizer)


res.to_csv(model_path + './sapbert_CE_may11/sapbert_CE_may11_testdata.csv')

In [ ]:
df.head()

In [ ]:
df.to_csv('test_with_clean_text.csv')

# to run on cohort pain sentences

In [ ]:
#df = pd.read_csv('./classified_as_pain_sapbert_CE.csv')
df = pd.read_csv('./test_data_pain.csv')

df.head()

In [ ]:
df = df.drop(columns='Unnamed: 0')
df.head()

In [ ]:
df.describe()

In [ ]:
df2 = df[:2000]
#df2 = df[2001:5000]
#df2 = df[5001:8000]
#df2 = df[8001:10000]
#df2 = df[10001:20000]
#df2 = df[20001:30000]
#df2 = df[30001:40000]
#df2 = df[40001:50000]
#df2 = df[50001:60000]
#df2 = df[60001:70000]
#df2 = df[70001:80000]
#df2 = df[80001:90000]
#df2 = df[90001:98940] #for anatomy
#df2 = df[90001:100000]
#df2 = df[100001:110000]
#df2 = df[110001:120000]
#df2 = df[120001:130000]
#df2 = df[130001:140000]
#df2 = df[140001:150000]
#df2 = df[150001:160000]
#df2 = df[160001:174042]
df2.to_csv('./anatomy_pain_sentences_for_classifier_90k_to_99k.csv')

df2.head()

In [ ]:
df2.describe()

In [ ]:
# RUN BERT ON LARGE FILE (EXEC LINE BY LINE) -- use this when running on new CRIS extraction -- update file names!!!
cpt=0
BERT_tokenizer = 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext' #'bert-base-uncased'
#BERT_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased') #'bert_base_uncased' #this seems to give a url error? and link that asks to sign in to huggingface
#model_path = data_path+'/bert_models/cognition_all_biobert_rel10'
model_path = './sapbert_model_dec13_second_script_crossentropy' #'./bert_base_model_nov14_second_script_notoverfit'
#names=['BrcId', 'CN_Doc_ID','doc_date', 'doc_type', 'kw', 'domain', 'start_kw', 'clean_text']#, 'age', 'ethnicitycleaned', 'Gender_ID']
names=['context', 'label']
for chunk in pd.read_csv('./test_data_pain.csv'
                         , chunksize=10000): #, names=names):
    cpt+=1
    print('chunk nb',cpt)
    res= load_and_run_BERT(sentences=chunk['context'] #['clean_text']
                   , trained_bert_model=model_path
                   , BERT_tokenizer=BERT_tokenizer)

    result = pd.concat([res.reset_index()[['preds','probs']]
                        , chunk.reset_index()[['context', 'label']]] #'BrcId', 'CN_Doc_ID','doc_date', 'doc_type', 'kw', 'domain', 'start_kw', 'clean_text']]]
                       , axis=1)
    print(result.head(1))
    header_bool=1 if cpt==1 else False
    result[['preds','probs']+names].to_csv('./test_data_pain_sapbert_CE_may11.csv', mode='a', header=header_bool)


In [ ]:
# this is taking about 1hr40 mins? for 1000 sentences

# BERT Classification 2

This will let you get confidence intervals

Another BERT training script - gives more details on loss, tokenisation, matthew's corr etc - and confidence intervals for f1, p and r

Training data

In [ ]:
training_data.loc[training_data.label == 0].sample(5)[['context', 'label']]

In [ ]:
training_data.loc[training_data.label == 1].sample(5)[['context', 'label']]

In [ ]:
# Get the lists of sentences and their labels.
sentences = training_data.context.values
labels = training_data.label.values

In [ ]:
type(labels)

Imports

In [ ]:
from transformers import BertTokenizer, AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
#tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext", do_lower_case=True)

#comparing tokenizers
bert_base_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
sapbert_tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext", do_lower_case=True)

Tokenise

In [ ]:
#testing some sentences out
sentences = ['e control of her anxiety much more effectively. we discussed regular practice of mindful breathing and attention control aimed at combating the catastrophic thinking and worry about symptoms of vulvodynia. we discussed pelvic floor relaxation exercises and importance of keeping up the practice. next session 04/01/18 10am. ---------------------------11 dec 2017 11:51 sylvia hejda-forde']

In [ ]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
#print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
#print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

# Print the sentence split into tokens.
print('Bert_base Tokenized: ', bert_base_tokenizer.tokenize(sentences[0]))


# Print the sentence split into tokens.
print('SAPBERT Tokenized: ', sapbert_tokenizer.tokenize(sentences[0]))

In [ ]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Split data

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

Load data

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 16 #tried 32 but 16 worked better for train and val loss

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

Load pretrained model

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig, AutoModelForSequenceClassification

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
#model = AutoModelForSequenceClassification.from_pretrained(
    #"bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    "cambridgeltl/SapBERT-from-PubMedBERT-fulltext",
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Print parameters

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 - changed back to 5e-5 to reduce overfitting - didnt work so tried 3e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
)
#loss_fn = nn.CrossEntropyLoss() #try this

Set up hyperparameters

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3 #changed from 4 because it looked like it was overfitting at 4 and 8 and 2 - changed back to 4 to test with diff learning rate

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Train and validate

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward`
        # function and pass down the arguments. The `forward` function is
        # documented here:
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result = model(b_input_ids,
                       token_type_ids=None,
                       attention_mask=b_input_mask,
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits
        #loss = loss_fn(logits,b_labels) #try this instead of loss=result.loss ?

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the
        # output values prior to applying an activation function like the
        # softmax.
        loss = result.loss
        logits = result.logits

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
#% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3])

plt.show()

plt.savefig('training_validation_loss.png')


In [ ]:
#performance on test set


# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(testing_data.shape[0]))

# Create sentence and label lists
sentences = testing_data.context.values
labels = testing_data.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.
batch_size = 16

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
#Evaluate on test set

# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables
predictions , true_labels = [], []

# Predict
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch

  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids,
                     token_type_ids=None,
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

In [ ]:
predictions

In [ ]:
true_labels

In [ ]:
print('Positive samples: %d of %d (%.2f%%)' % (testing_data.label.sum(), len(testing_data.label), (testing_data.label.sum() / len(testing_data.label) * 100.0)))

Confidence Interval for BERT model! - run this after the MCC stuff

In [ ]:
###insert functions for f1_conf_int, precision and recall and per label then run

In [ ]:
f1_conf_int(flat_true_labels, flat_predictions) #gives f1 mean, ci lower, ci upper, std

In [ ]:
class_0_f1_conf_int(flat_true_labels, flat_predictions)

In [ ]:
class_1_f1_conf_int(flat_true_labels, flat_predictions)

In [ ]:
f1_score(flat_true_labels, flat_predictions)

In [ ]:
print(classification_report(flat_true_labels, flat_predictions))

In [ ]:
precision_conf_int(flat_true_labels, flat_predictions)

In [ ]:
recall_conf_int(flat_true_labels, flat_predictions)

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):

  # The predictions for this batch are a 2-column ndarray (one column for "0"
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

  # Calculate and store the coef for this batch.
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)
  matthews_set.append(matthews)

In [ ]:
# Create a barplot showing the MCC score for each batch of test samples.
ax = sns.barplot(x=list(range(len(matthews_set))), y=matthews_set, ci=None)

plt.title('MCC Score per Batch')
plt.ylabel('MCC Score (-1 to +1)')
plt.xlabel('Batch #')

plt.savefig('MCC_bert_base.png')
plt.show()

In [ ]:
# Combine the results across all batches.
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

In [ ]:
#saving model - not done

import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './bert_base_model_nov14_second_script_notoverfit'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
#torch.save(args, os.path.join(output_dir, 'training_args.bin'))

In [ ]:
#file sizes

!ls -l --block-size=K './'

In [ ]:
!ls -l --block-size=M './pytorch_model.bin'

In [ ]:
model = torch.load('./pytorch_model.bin')

Need F1 score as well

In [ ]:
import os
from collections import Counter
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import StratifiedKFold
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset
from tqdm import trange
from transformers import BertForSequenceClassification, BertTokenizerFast, AdamW, AutoModel, AutoModelForSequenceClassification, AutoTokenizer  #BertTokenizer #AutoModel was missing, added in - JC

In [ ]:
# run k-fold validation
kf = BERT_KFOLD(sentences=data_for_classifier['text'].to_list(), labels=data_for_classifier['label'].to_list(), n_splits=5, BERT_tokenizer=BERT_tokenizer, n_epochs=1,
                random_state=666)
print(kf['stats'], '\n\n', kf['stats_classes'])

# GPT-2

https://colab.research.google.com/github/gmihaila/ml_things/blob/master/notebooks/pytorch/gpt2_finetune_classification.ipynb#scrollTo=EDEubgJIt23C

In [ ]:
# Install helper functions.
!pip install -q git+https://github.com/gmihaila/ml_things.git

In [ ]:
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW,
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)

# Set seed for reproducibility.
set_seed(123)

# Number of training epochs (authors on fine-tuning Bert recommend between 2 and 4).
epochs = 4

# Number of batches - depending on the max sequence length and GPU memory.
# For 512 sequence length batch of 10 works without cuda memory issues.
# For small sequence length can try batch of 32 or higher.
batch_size = 32

# Pad or truncate text sequences to a specific length
# if `None` it will use maximum sequence of word piece tokens allowed by model.
max_length = 60

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Name of transformers model - will use already pretrained model.
# Path of transformer model - will load your own model from local disk.
model_name_or_path = 'gpt2'

# Dictionary of labels and their id - this will be used to convert.
# String labels to number ids.
labels_ids = {'not relevant': 0, 'relevant': 1}

# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)

In [ ]:

class Gpt2ClassificationCollator(object):
    r"""
    Data Collator used for GPT2 in a classificaiton rask.

    It uses a given tokenizer and label encoder to convert any text and labels to numbers that
    can go straight into a GPT2 model.

    This class is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed
    straight into the model - `model(**batch)`.

    Arguments:

      use_tokenizer (:obj:`transformers.tokenization_?`):
          Transformer type tokenizer used to process raw text into numbers.

      labels_ids (:obj:`dict`):
          Dictionary to encode any labels names into numbers. Keys map to
          labels names and Values map to number associated to those labels.

      max_sequence_len (:obj:`int`, `optional`)
          Value to indicate the maximum desired sequence to truncate or pad text
          sequences. If no value is passed it will used maximum sequence size
          supported by the tokenizer and model.

    """

    def __init__(self, use_tokenizer, labels_encoder, max_sequence_len=None):

        # Tokenizer to be used inside the class.
        self.use_tokenizer = use_tokenizer
        # Check max sequence length.
        self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len
        # Label encoder used inside the class.
        self.labels_encoder = labels_encoder

        return

    def __call__(self, sequences):
        r"""
        This function allowes the class objesct to be used as a function call.
        Sine the PyTorch DataLoader needs a collator function, I can use this
        class as a function.

        Arguments:

          item (:obj:`list`):
              List of texts and labels.

        Returns:
          :obj:`Dict[str, object]`: Dictionary of inputs that feed into the model.
          It holddes the statement `model(**Returned Dictionary)`.
        """

        # Get all texts from sequences list.
        texts = [sequence['text'] for sequence in sequences]
        # Get all labels from sequences list.
        labels = [sequence['label'] for sequence in sequences]
        # Encode all labels using label encoder.
        labels = [self.labels_encoder[label] for label in labels]
        # Call tokenizer on all texts to convert into tensors of numbers with
        # appropriate padding.
        inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len)
        # Update the inputs with the associated encoded labels as tensor.
        inputs.update({'labels':torch.tensor(labels)})

        return inputs


def train(dataloader, optimizer_, scheduler_, device_):
  r"""
  Train pytorch model on a single pass through the data loader.

  It will use the global variable `model` which is the transformer model
  loaded on `_device` that we want to train on.

  This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed
    straight into the model - `model(**batch)`.

  Arguments:

      dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

      optimizer_ (:obj:`transformers.optimization.AdamW`):
          Optimizer used for training.

      scheduler_ (:obj:`torch.optim.lr_scheduler.LambdaLR`):
          PyTorch scheduler.

      device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.

  Returns:

      :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss].
  """

  # Use global variable for model.
  global model

  # Tracking variables.
  predictions_labels = []
  true_labels = []
  # Total loss for this epoch.
  total_loss = 0

  # Put the model into training mode.
  model.train()

  # For each batch of training data...
  for batch in tqdm(dataloader, total=len(dataloader)):

    # Add original labels - use later for evaluation.
    true_labels += batch['labels'].numpy().flatten().tolist()

    # move batch to device
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

    # Always clear any previously calculated gradients before performing a
    # backward pass.
    model.zero_grad()

    # Perform a forward pass (evaluate the model on this training batch).
    # This will return the loss (rather than the model output) because we
    # have provided the `labels`.
    # The documentation for this a bert model function is here:
    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    outputs = model(**batch)

    # The call to `model` always returns a tuple, so we need to pull the
    # loss value out of the tuple along with the logits. We will use logits
    # later to calculate training accuracy.
    loss, logits = outputs[:2]

    # Accumulate the training loss over all of the batches so that we can
    # calculate the average loss at the end. `loss` is a Tensor containing a
    # single value; the `.item()` function just returns the Python value
    # from the tensor.
    total_loss += loss.item()

    # Perform a backward pass to calculate the gradients.
    loss.backward()

    # Clip the norm of the gradients to 1.0.
    # This is to help prevent the "exploding gradients" problem.
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # Update parameters and take a step using the computed gradient.
    # The optimizer dictates the "update rule"--how the parameters are
    # modified based on their gradients, the learning rate, etc.
    optimizer_.step()

    # Update the learning rate.
    scheduler_.step()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    # Convert these logits to list of predicted labels values.
    predictions_labels += logits.argmax(axis=-1).flatten().tolist()

  # Calculate the average loss over the training data.
  avg_epoch_loss = total_loss / len(dataloader)

  # Return all true labels and prediction for future evaluations.
  return true_labels, predictions_labels, avg_epoch_loss



def validation(dataloader, device_):
  r"""Validation function to evaluate model performance on a
  separate set of data.

  This function will return the true and predicted labels so we can use later
  to evaluate the model's performance.

  This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed
    straight into the model - `model(**batch)`.

  Arguments:

    dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

    device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.

  Returns:

    :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss]
  """

  # Use global variable for model.
  global model

  # Tracking variables
  predictions_labels = []
  true_labels = []
  #total loss for this epoch.
  total_loss = 0

  # Put the model in evaluation mode--the dropout layers behave differently
  # during evaluation.
  model.eval()

  # Evaluate data for one epoch
  for batch in tqdm(dataloader, total=len(dataloader)):

    # add original labels
    true_labels += batch['labels'].numpy().flatten().tolist()

    # move batch to device
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(**batch)

        # The call to `model` always returns a tuple, so we need to pull the
        # loss value out of the tuple along with the logits. We will use logits
        # later to to calculate training accuracy.
        loss, logits = outputs[:2]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_loss += loss.item()

        # get predicitons to list
        predict_content = logits.argmax(axis=-1).flatten().tolist()

        # update list
        predictions_labels += predict_content

  # Calculate the average loss over the training data.
  avg_epoch_loss = total_loss / len(dataloader)

  # Return all true labels and prediciton for future evaluations.
  return true_labels, predictions_labels, avg_epoch_loss

In [ ]:
# Get model configuration.
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config)

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

In [ ]:
import pandas as pd

df = pd.read_csv('gold_std_clean.csv')

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={'context': 'text'})
df.head()

In [ ]:
# Mapping dictionary for label values
label_mapping = {0: 'not relevant', 1: 'relevant'}

# Replace label values using the mapping dictionary
df['label'] = df['label'].map(label_mapping)

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
#Creating a dataframe with 80% values of original dataframe
train_df = df.sample(frac = 0.8)

#Creating dataframe with rest of the 20% values
test_df = df.drop(train_df.index)

In [ ]:
print('train dataset: ', len(train_df))
print('test dataset: ', len(train_df))

In [ ]:
train_df.to_csv('gold_std_clean_train.csv')
train_df.to_csv('gold_std_clean_test.csv')

In [ ]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

'''
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, index):
        row = self.dataframe.iloc[index].to_numpy()
        features = row[1:]
        label = row[2]
        return features, label

    def __len__(self):
        return len(self.dataframe)


'''

In [ ]:
'''
train_dataset = CustomDataset(dataframe=train_df)
test_dataset = CustomDataset(dataframe=test_df)
'''

In [ ]:

'''
# Assuming you already have a dataframe called `df` with columns 'text' and 'label' - make sure you split them into train and test before this so you can run it for both
# and get 2 dictionaries - one for train and one for validation. Name them appropriately so you don't have to make much changes to the code.

# Create empty lists for texts and labels
texts = []
labels = []

# Iterate over each row in the dataframe
for index, row in train_df.iterrows():
    text = row['context']
    label = row['label']

    # Create a dictionary for the current row
    example = {'text': text, 'label': label}

    # Append the example to the lists
    texts.append(example['text'])
    labels.append(example['label'])

# Create a dictionary with 'texts' and 'labels' lists
train_dict = {'texts': texts, 'labels': labels}

'''

In [ ]:
#train_dict['texts'][0]

In [ ]:
#train_dict['labels'][0]

In [ ]:
class MovieReviewsDataset(Dataset):
    def __init__(self, dataframe, use_tokenizer):
        self.texts = dataframe['text'].tolist()
        self.labels = dataframe['label'].tolist()
        self.n_examples = len(self.labels)

    def __len__(self):
        return self.n_examples

    def __getitem__(self, item):
        return {'text': self.texts[item], 'label': self.labels[item]}


In [ ]:
# Create data collator to encode text and labels into numbers.
gpt2_classificaiton_collator = Gpt2ClassificationCollator(use_tokenizer=tokenizer,
                                                          labels_encoder=labels_ids,
                                                          max_sequence_len=max_length)

print('Dealing with Train...')
# Create pytorch dataset. -- modify this for my use case
train_dataset = MovieReviewsDataset(train_df,
                               use_tokenizer=tokenizer)
print('Created `train_dataset` with %d examples!'%len(train_dataset))


# Move train dataset into dataloader.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=gpt2_classificaiton_collator)
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))

print()

print('Dealing with Validation...')
# Create pytorch dataset. -- modify this for my use case
valid_dataset =  MovieReviewsDataset(test_df,
                               use_tokenizer=tokenizer)
print('Created `valid_dataset` with %d examples!'%len(valid_dataset))

# Move test dataset into dataloader.
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=gpt2_classificaiton_collator)
print('Created `eval_dataloader` with %d batches!'%len(valid_dataloader))

In [ ]:
type(train_dataset)

In [ ]:
type(train_dataloader)

In [ ]:
type(optimizer)

In [ ]:
type(scheduler)

In [ ]:
type(device)

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )

# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [ ]:
# Store the average loss after each epoch so we can plot them.
all_loss = {'train_loss':[], 'val_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}

In [ ]:
# Loop through each epoch.
print('Epoch')
for epoch in tqdm(range(epochs)):
  print()
  print('Training on batches...')
  # Perform one full pass over the training set.
  train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
  train_acc = accuracy_score(train_labels, train_predict)

  # Get prediction form model on validation data.
  print('Validation on batches...')
  valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
  val_acc = accuracy_score(valid_labels, valid_predict)

  # Print loss and accuracy values to see how training evolves.
  print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, val_loss, train_acc, val_acc))
  print()

  # Store the loss value for plotting the learning curve.
  all_loss['train_loss'].append(train_loss)
  all_loss['val_loss'].append(val_loss)
  all_acc['train_acc'].append(train_acc)
  all_acc['val_acc'].append(val_acc)

# Plot loss curves.
plot_dict(all_loss, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

# Plot accuracy curves.
plot_dict(all_acc, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

In [ ]:
# Evaluate
# Get prediction form model on validation data. This is where you should use
# your test data.
true_labels, predictions_labels, avg_epoch_loss = validation(valid_dataloader, device)

# Create the evaluation report.
evaluation_report = classification_report(true_labels, predictions_labels, labels=list(labels_ids.values()), target_names=list(labels_ids.keys()))
# Show the evaluation report.
print(evaluation_report)

# Plot confusion matrix.
plot_confusion_matrix(y_true=true_labels, y_pred=predictions_labels,
                      classes=list(labels_ids.keys()), normalize=True,
                      magnify=0.1,
                      );

# Non-BERT Classifiers (Ignore)

Non-BERT

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
try:
    import xlrd
except ImportError as e:
    !pip install xlrd
    import xlrd

import numpy as np

import warnings; warnings.simplefilter('ignore')


In [ ]:
# We'll use scikit-learn for the classification algorithms.
# https://scikit-learn.org/stable/

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [ ]:
## sklearn also has some nice funtions for representations

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## and for evaluation
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, precision_score, recall_score #JC - look at More non-BERT section
from sklearn.metrics import roc_curve, auc, roc_auc_score #JC - look at More non-BERT section

In [ ]:
!pip install seaborn
import seaborn as sns

In [ ]:
## Since we're working with text, we might need to tokenize for some of these representations.
# We'll use nltk here, but there are other nlp packages available for this
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# You have also learnt about embedding representations. These can also be used for classification.
# We will use a library called Zeugma, which allows using pre-trained embedding models
#Zeugma library: https://github.com/nkthiebaut/zeugma

try:
    from zeugma.embeddings import EmbeddingTransformer
except ImportError as e:
    !pip install zeugma
    !pip install theano
    from zeugma.embeddings import EmbeddingTransformer

from datetime import datetime
print(datetime.now())

In [ ]:
training_data['label'].value_counts()

In [ ]:
sns.countplot(training_data['label'])

In [ ]:
trainingdata, valdata, testdata = np.split(data_for_classifier.sample(frac=1), [int(.6*len(data_for_classifier)), int(.8*len(data_for_classifier))])

In [ ]:
print("Training data: ", trainingdata.shape)
print("Test data: ", testdata.shape)
print("Validation data: ", valdata.shape)

In [ ]:
#training_data_example = training_data['text'].tolist()[0]
training_data_example = trainingdata['context'].tolist()[12]
print(training_data_example)

# BoW Representation

BoW Representation

In [ ]:
first_vectorizer = CountVectorizer(ngram_range=(1,1), stop_words=None,
                             tokenizer=word_tokenize, max_features=500)
#first_vectorizer.fit(training_data['text'].tolist())
first_vectorizer.fit(training_data['context'].values.astype('U').tolist())
#first_vectorizer.fit(trainingdata['text'].values.astype('U').tolist())
#first_fit_transformed_data = first_vectorizer.fit_transform(training_data['text'])
first_fit_transformed_data = first_vectorizer.fit_transform(training_data['context'].values.astype('U'))

In [ ]:
first_transformed_data = first_vectorizer.transform([training_data['context'].tolist()[231]])
print (first_transformed_data)

In [ ]:
print (first_vectorizer.get_feature_names()[377])

In [ ]:
print(first_fit_transformed_data.shape)
print ('Amount of Non-Zero occurences: ', first_fit_transformed_data.nnz)

Build a classifier with this feature representation

K-nearest Neighbour

In [ ]:
kneighbour_classifier = KNeighborsClassifier().fit(first_fit_transformed_data, training_data['label'])

In [ ]:
## We need to transform test data to the same representation
first_fit_transformed_testdata = first_vectorizer.transform(testing_data['context'].values.astype('U'))

In [ ]:
first_fit_transformed_testdata
kneighbour_predicted = kneighbour_classifier.predict(first_fit_transformed_testdata)
kneighbour_predicted

In [ ]:
print(metrics.classification_report(testing_data['label'], kneighbour_predicted, target_names=set(testing_data['label'].values.astype('U').tolist())))

We can employ n-fold cross-validation on the training data to experiment with different representations, parameters, and classifiers.

In [ ]:
kneighbour_classifier = KNeighborsClassifier().fit(first_fit_transformed_data, training_data['label'])
scoring = ['precision_macro', 'recall_macro','precision_micro','recall_micro', 'f1_micro', 'f1_macro']
scores = cross_validate(kneighbour_classifier, first_fit_transformed_data, training_data['label'], scoring=scoring, cv=10, return_train_score=False)
scoresdf = pd.DataFrame(scores)
scoring = ['test_precision_macro', 'test_recall_macro','test_precision_micro','test_recall_micro', 'test_f1_micro', 'test_f1_macro']
bp = scoresdf.boxplot(column=scoring, grid=False, rot=45,)
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('K nearest neighbour, count vectorizer')
plt.show()

In [ ]:
scoresdf.head()

Random Forest Classifier

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0).fit(first_fit_transformed_data, training_data['label'])
scoring = ['precision_macro', 'recall_macro','precision_micro','recall_micro', 'f1_micro', 'f1_macro']
scores = cross_validate(rf_classifier, first_fit_transformed_data, training_data['label'], scoring=scoring, cv=10, return_train_score=False)
scoresdf = pd.DataFrame(scores)
scoring = ['test_precision_macro', 'test_recall_macro','test_precision_micro','test_recall_micro', 'test_f1_micro', 'test_f1_macro']
bp = scoresdf.boxplot(column=scoring, grid=False, rot=45,)
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('Random forest, count vectorizer')
plt.show()

# TF-IDF Representation

Tf-idf representation

In [ ]:
stopWords = set(stopwords.words('english'))
tfidf_vect = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopWords)
tfidf_vect.fit(training_data['context'])
second_fit_transformed_data =  tfidf_vect.transform(training_data['context'])
second_fit_transformed_data

Use this with Multinomial Naive Bayes classifier

In [ ]:
kneighbour_classifier = KNeighborsClassifier().fit(second_fit_transformed_data, training_data['label'])
scoring = ['precision_macro', 'recall_macro','precision_micro','recall_micro', 'f1_micro', 'f1_macro']
scores = cross_validate(kneighbour_classifier, second_fit_transformed_data, training_data['label'], scoring=scoring, cv=10, return_train_score=False)
scoresdf = pd.DataFrame(scores)
scoring = ['test_precision_macro', 'test_recall_macro','test_precision_micro','test_recall_micro', 'test_f1_micro', 'test_f1_macro']
bp = scoresdf.boxplot(column=scoring, grid=False, rot=45,)
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('K nearest neighbour, tf-idf vectorizer')
plt.show()

# Embedding Representations

Embedding Representations

In [ ]:
glove = EmbeddingTransformer('glove')

In [ ]:
glove_transformed_training_data = glove.transform(training_data['context'])

In [ ]:
glove_transformed_training_data

Build a classifier with this representation and evaluate with 10-fold cross-validation

In [ ]:
kneighbour_classifier = KNeighborsClassifier().fit(glove_transformed_training_data, training_data['label'])
scoring = ['precision_macro', 'recall_macro','precision_micro','recall_micro', 'f1_micro', 'f1_macro']
scores = cross_validate(kneighbour_classifier, glove_transformed_training_data, training_data['label'], scoring=scoring, cv=10, return_train_score=False)
scoresdf = pd.DataFrame(scores)
scoring = ['test_precision_macro', 'test_recall_macro','test_precision_micro','test_recall_micro', 'test_f1_micro', 'test_f1_macro']
bp = scoresdf.boxplot(column=scoring, grid=False, rot=45,)
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('K nearest neighbour, pre-trained embeddings')
plt.show()

# Different configurations in one go

Some different configurations all in one go

In [ ]:

representations = {}

vectorizer = CountVectorizer(ngram_range=(1,1), stop_words=None,
                             tokenizer=word_tokenize, max_features=500)
xtrain_countvect = vectorizer.fit_transform(training_data['context'])
representations['CountVectorizer'] = xtrain_countvect

tfidf_vect = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopWords)
tfidf_vect.fit(training_data['context'])
xtrain_tfidf =  tfidf_vect.transform(training_data['context'])
representations['TfidfVectorizer'] = xtrain_tfidf

x_train_glove = glove.transform(training_data['context'])
representations['pretrained_glove'] = x_train_glove



CV = 10

classifier_models = [
        RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
        DecisionTreeClassifier(),
        SVC(),
        LinearSVC(multi_class='ovr', C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      penalty='l2', random_state=0, tol=1e-05, verbose=0),
        SGDClassifier(),
        LogisticRegression(random_state=0),
        KNeighborsClassifier(),
]

cv_df = pd.DataFrame(index=range(CV * (len(classifier_models)*len(representations))))
entries = []


for representation, transformed_vector in representations.items():
    score = 'f1_micro'
    for model in classifier_models:
      model_name = model.__class__.__name__+'_'+representation
      accuracies = cross_val_score(model, transformed_vector, training_data['label'], scoring=score, cv=CV)
      for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', score])
bp = cv_df.boxplot(by='model_name', column=[score], grid=False, rot=90, figsize=(12,8))
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('10-fold cross validation results')
plt.show()

# More experiments

In [ ]:

## First step: Transform your training and test data to your chosen representation.

## choose a representation: CountVectorizer or TfidfVectorizer, or embeddings

chosen_representation = tfidf_vect

## transform the training data
transformed_training_data = chosen_representation.transform(training_data['context'])

## transform the test data
transformed_test_data = chosen_representation.transform(testing_data['context'])

## Second step: Create a classifier - the one you think gave best results when experimenting with cross-validation


chosen_classifier = LinearSVC(multi_class='ovr', C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      penalty='l2', random_state=0, tol=1e-05, verbose=0)

## train the classifier on the training data
chosen_classifier.fit(transformed_training_data, training_data['label'])

## predict labels on the test data
predicted = chosen_classifier.predict(transformed_test_data)

## what results do you get? Note that you can look at both macro and micro scores!
print(metrics.classification_report(testing_data['label'], predicted, target_names=set(testing_data['label'].values.astype('U').tolist())))

# Confidence Intervals

Get confidence intervals - https://sebastianraschka.com/blog/2022/confidence-intervals-for-ml.html

code - https://gist.github.com/roncho12/60178f12ea4c3a74764fd645c6f2fe13

https://towardsdatascience.com/how-to-add-confidence-intervals-to-any-model-7bbb9f80fd9c


https://sebastianraschka.com/blog/2022/confidence-intervals-for-ml.html


In [ ]:
import scipy.stats

Bootstrapping

In [ ]:
#From Diana

def f1_conf_int(target, predict, bootstrap_n=500):
    n = len(target)
    f1 = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace=True)
        tp = pd.DataFrame({"t":target, "p":predict})
        tp_bootstrap = tp.iloc[i,:]
        f1[bs] = (f1_score(tp_bootstrap["t"], tp_bootstrap["p"]))
    res = [f1.mean(), np.quantile(f1,0.025), np.quantile(f1,0.975), f1.std()] #use these numbers for 95% CI
    return res

In [ ]:
def precision_conf_int(target, predict, bootstrap_n=500):
    n = len(target)
    precision = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace=True)
        tp = pd.DataFrame({"t":target, "p":predict})
        tp_bootstrap = tp.iloc[i,:]
        precision[bs] = (precision_score(tp_bootstrap["t"], tp_bootstrap["p"]))
    res = [precision.mean(), np.quantile(precision,0.025), np.quantile(precision,0.975), precision.std()]#use these numbers for 95% CI
    return res

In [ ]:
def recall_conf_int(target, predict, bootstrap_n=500):
    n = len(target)
    recall = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace=True)
        tp = pd.DataFrame({"t":target, "p":predict})
        tp_bootstrap = tp.iloc[i,:]
        recall[bs] = (recall_score(tp_bootstrap["t"], tp_bootstrap["p"]))
    res = [recall.mean(), np.quantile(recall,0.025), np.quantile(recall,0.975), recall.std()]#use these numbers for 95% CI
    return res

In [ ]:
bootstrap_n=100
bootstrap_n

In [ ]:
recall = np.zeros(bootstrap_n)
recall

In [ ]:
np.quantile(recall,0.94)

In [ ]:
100*0.06

# Predict on unseen random text

Predict unseen random text

In [ ]:
new_text = 'Patient with severe depression and heart pain.'
testX = chosen_representation.transform([new_text])
predicted = chosen_classifier.predict(testX)
print(predicted)

In [ ]:
new_text = '5-year old girl with asthma.'
testX = chosen_representation.transform([new_text])
predicted = chosen_classifier.predict(testX)
print(predicted)

In [ ]:
new_text = 'Her asthma was getting better'
testX = chosen_representation.transform([new_text])
predicted = chosen_classifier.predict(testX)
print(predicted)

In [ ]:
new_text = 'that movie was amazing.'
testX = chosen_representation.transform([new_text])
predicted = chosen_classifier.predict(testX)
print(predicted)

# More non-BERT classifiers - use these

https://github.com/vijayaiitk/NLP-text-classification-model/blob/main/NLP%20text%20classification%20model%20Github.ipynb


https://medium.com/analytics-vidhya/nlp-tutorial-for-text-classification-in-python-8f19cd17b49e#:~:text=Text%20classification%20is%20one%20of,in%20a%20cost%2Deffective%20manner.

In [ ]:
!pip install nltk

In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC #JC
from sklearn.ensemble import RandomForestClassifier #JC
from sklearn.neighbors import KNeighborsClassifier #JC
from sklearn.naive_bayes import GaussianNB #JC
from sklearn.tree import DecisionTreeClassifier #JC
from sklearn.svm import SVC #JC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer #JC

#for word embedding
import gensim
from gensim.models import Word2Vec #Word2Vec is mostly used for huge datasets

In [ ]:
#you can download the data from https://www.kaggle.com/c/nlp-getting-started/data
import os
#os.chdir('/Users/ranivija/Desktop/')
#df_train=pd.read_csv('train.csv')
#df_train=pd.read_csv('gdrive/My Drive/sample_size/5000_data_for_classifier.csv')#5000 docs

df_train = training_data
print(df_train.shape)
df_train.head()

In [ ]:
# CLASS DISTRIBUTION
#if dataset is balanced or not
#x=df_train['target'].value_counts()
x=df_train['label'].value_counts()
print(x)
sns.barplot(x.index,x)

In [ ]:
#Missing values
df_train.isna().sum()

In [ ]:
#1. WORD-COUNT
df_train['word_count'] = df_train['context'].apply(lambda x: len(str(x).split()))
#print(df_train[df_train['target']==1]['word_count'].mean()) #Disaster tweets
#print(df_train[df_train['target']==0]['word_count'].mean()) #Non-Disaster tweets
#Disaster tweets are more wordy than the non-disaster tweets
print("Relevant word count: ", df_train[df_train['label']==1]['word_count'].mean()) #HTN
print("Not relevant word count: ", df_train[df_train['label']==0]['word_count'].mean()) #Non-HTN

#2. CHARACTER-COUNT
df_train['char_count'] = df_train['context'].apply(lambda x: len(str(x)))
#print(df_train[df_train['target']==1]['char_count'].mean()) #Disaster tweets
#print(df_train[df_train['target']==0]['char_count'].mean()) #Non-Disaster tweets
#Disaster tweets are longer than the non-disaster tweets
print("Relevant char count: ",df_train[df_train['label']==1]['char_count'].mean()) #HTN
print("Not relevant char count: ",df_train[df_train['label']==0]['char_count'].mean()) #Non-HTN

#3. UNIQUE WORD-COUNT
df_train['unique_word_count'] = df_train['context'].apply(lambda x: len(set(str(x).split())))
#print(df_train[df_train['target']==1]['unique_word_count'].mean()) #Disaster tweets
#print(df_train[df_train['target']==0]['unique_word_count'].mean()) #Non-Disaster tweets
print("Relevant unique word count: ",df_train[df_train['label']==1]['unique_word_count'].mean()) #HTN
print("Not relevant unique word count: ",df_train[df_train['label']==0]['unique_word_count'].mean()) #Non-HTN

In [ ]:
#Plotting word-count per tweet
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,4))
#train_words=df_train[df_train['target']==1]['word_count']
train_words=df_train[df_train['label']==1]['word_count']
ax1.hist(train_words,color='red')
#ax1.set_title('Disaster Tweets')
ax1.set_title('Relevant')
#train_words=df_train[df_train['target']==0]['word_count']
train_words=df_train[df_train['label']==0]['word_count']
ax2.hist(train_words,color='green')
#ax2.set_title('Non-Disaster Tweets')
ax2.set_title('Not relevant')
#fig.suptitle('Words per tweet')
fig.suptitle('Words per document')
plt.show()

In [ ]:
#1. Common text preprocessing
text = "   This is a message to be cleaned. It may involve some things like: <br>, ?, :, ''  adjacent spaces and tabs     .  "

#convert to lowercase and remove punctuations and characters and then strip
def preprocess(text):
    text = text.lower() #lowercase text
    text=text.strip()  #get rid of leading/trailing whitespace
    text=re.compile('<.*?>').sub('', text) #Remove HTML tags/markups
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  #Replace punctuation with space. Careful since punctuation can sometime be useful
    text = re.sub('\s+', ' ', text)  #Remove extra space and tabs
    text = re.sub(r'\[[0-9]*\]',' ',text) #[0-9] matches any digit (0 to 10000...)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) #matches any digit from 0 to 100000..., \D matches non-digits
    text = re.sub(r'\s+',' ',text) #\s matches any whitespace, \s+ matches multiple whitespace, \S matches non-whitespace

    return text

text=preprocess(text)
print(text)  #text is a string

In [ ]:
#3. LEXICON-BASED TEXT PROCESSING EXAMPLES

#1. STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

text=stopword(text)
print(text)

#2. STEMMING

# Initialize the stemmer
snow = SnowballStemmer('english')
def stemming(string):
    a=[snow.stem(i) for i in word_tokenize(string) ]
    return " ".join(a)
text=stemming(text)
print(text)

#3. LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
# Full list is available here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

text = lemmatizer(text)
print(text)

In [ ]:
#FINAL PREPROCESSING
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

df_train['clean_text'] = df_train['context'].apply(lambda x: finalpreprocess(x)) #context instead of text
df_train=df_train.drop(columns=['word_count','char_count','unique_word_count'])
df_train.head()

In [ ]:
# create Word2vec model
#here words_f should be a list containing words from each document. say 1st row of the list is words from the 1st document/sentence
#length of words_f is number of documents/sentences in your dataset
df_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['clean_text']] #convert preprocessed sentence to tokenized sentence
model = Word2Vec(df_train['clean_text_tok'],min_count=1)  #min_count=1 means word should be present at least across all documents,
#if min_count=2 means if the word is present less than 2 times across all the documents then we shouldn't consider it

##The index2word attribute has been replaced by index_to_key since Gensim 4.0.0.
##model.wv.syn0 informs about " DeprecationWarning: Call to deprecated syn0 (Attribute will be removed in 4.0.0, use self.wv.vectors instead)." -- i used model.wv.vectors
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))  #combination of word and its vector

#for converting sentence to vectors/numbers from word vectors result by Word2Vec
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION

# Input: "reviewText", "rating" and "time"
# Target: "log_votes"
X_train, X_val, y_train, y_val = train_test_split(df_train["clean_text"],
                                                  #df_train["target"],
                                                  df_train["label"],
                                                  test_size=0.2,
                                                  shuffle=True)
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  #for word2vec
X_val_tok= [nltk.word_tokenize(i) for i in X_val]      #for word2vec

#TF-IDF
# Convert x_train to vector since model can only run on numbers and not words- Fit and transform
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) #tfidf runs on non-tokenized sentences unlike word2vec
# Only transform x_test (not fit and transform)
X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) #Don't fit() your TfidfVectorizer to your test data: it will
#change the word-indexes & weights to match test data. Rather, fit on the training data, then use the same train-data-
#fit model on the test data, to reflect the fact you're analyzing the test data only based on what was learned without
#it, and the have compatible


#Word2vec
# Fit and transform
modelw = MeanEmbeddingVectorizer(w2v)
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_val_tok)

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)

lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_val_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]

print('Logistic regression + tf-idf')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
f1_conf_int(y_val,y_predict) #gives f1 mean, ci lower, ci upper, std

In [ ]:
list1 = ['mean', 'ci_lower', 'ci_upper' , 'std']

In [ ]:
lrtfidf_f1 = pd.DataFrame(f1_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])
lrtfidf_f1

In [ ]:
precision_conf_int(y_val, y_predict)

In [ ]:
lrtfidf_p = pd.DataFrame(precision_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])
lrtfidf_p

In [ ]:
recall_conf_int(y_val, y_predict)

lrtfidf_r = pd.DataFrame(recall_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])
lrtfidf_r

In [ ]:
lr_tfidf_report = classification_report(y_val,y_predict, output_dict=True)
lrtfidf = pd.DataFrame(lr_tfidf_report).transpose()
lrtfidf.head()

In [ ]:
auc_list=[]

auc_list.append(roc_auc)
lrtfidf_auc = pd.DataFrame(auc_list, columns=['auc'])
lrtfidf_auc

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
#It's a probabilistic classifier that makes use of Bayes' Theorem, a rule that uses probability to make predictions based on prior knowledge of conditions that might be related. This algorithm is the most suitable for such large dataset as it considers each feature independently, calculates the probability of each category, and then predicts the category with the highest probability.


nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_val_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]

print('NB + tf-idf')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
f1_conf_int(y_val,y_predict) #gives f1 mean, ci lower, ci upper, std

In [ ]:
nbtfidf_f1 = pd.DataFrame(f1_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

nbtfidf_p = pd.DataFrame(precision_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

nbtfidf_r = pd.DataFrame(recall_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

nb_tfidf_report = classification_report(y_val,y_predict, output_dict=True)
nbtfidf = pd.DataFrame(nb_tfidf_report).transpose()

auc_list=[]

auc_list.append(roc_auc)
nbtfidf_auc = pd.DataFrame(auc_list, columns=['auc'])


In [ ]:
#FITTING THE CLASSIFICATION MODEL using Random Forest (tf-idf) ---- try

#rf_tfidf = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
rf_tfidf = RandomForestClassifier()
rf_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = rf_tfidf.predict(X_val_vectors_tfidf)
y_prob = rf_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]

print('RF + tf-idf')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
f1_conf_int(y_val,y_predict) #gives f1 mean, ci lower, ci upper, std

In [ ]:
rftfidf_f1 = pd.DataFrame(f1_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

rftfidf_p = pd.DataFrame(precision_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

rftfidf_r = pd.DataFrame(recall_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

rf_tfidf_report = classification_report(y_val,y_predict, output_dict=True)
rftfidf = pd.DataFrame(rf_tfidf_report).transpose()

auc_list=[]

auc_list.append(roc_auc)
rftfidf_auc = pd.DataFrame(auc_list, columns=['auc'])

In [ ]:
import _pickle as cPickle
import pandas as pd
import numpy as np
import joblib

In [ ]:
import _pickle as cPickle
#import cPickle

with open('RF_tfidf_may4', 'wb') as f:
    cPickle.dump(rf_tfidf, f)


# in your prediction file

#with open('S:/AchlysShared/BRC_CRIS/Jaya Chaturvedi/13_Pain/from_gpu_vm_classifier_and_ampligraph/ampligraph/KNclf', 'rb') as f:
#    kn_model = cPickle.load(f)

In [ ]:
import joblib

# save
joblib.dump(rf_tfidf, "RF_tfidf_may4.joblib")

# load
#loaded_rf = joblib.load("S:/AchlysShared/BRC_CRIS/Jaya Chaturvedi/13_Pain/from_gpu_vm_classifier_and_ampligraph/ampligraph/randomforestclf.joblib")

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Decision Tree (tf-idf) ---- try

dt_tfidf = DecisionTreeClassifier()
dt_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = dt_tfidf.predict(X_val_vectors_tfidf)
y_prob = dt_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]

print('Decision Tree + tf-idf')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
f1_conf_int(y_val,y_predict) #gives f1 mean, ci lower, ci upper, std

In [ ]:
dttfidf_f1 = pd.DataFrame(f1_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

dttfidf_p = pd.DataFrame(precision_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

dttfidf_r = pd.DataFrame(recall_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

dt_tfidf_report = classification_report(y_val,y_predict, output_dict=True)
dttfidf = pd.DataFrame(dt_tfidf_report).transpose()

auc_list=[]

auc_list.append(roc_auc)
dttfidf_auc = pd.DataFrame(auc_list, columns=['auc'])

In [ ]:
#FITTING THE CLASSIFICATION MODEL using SVC (tf-idf) ---- try

svc_tfidf = SVC(probability=True)
svc_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = svc_tfidf.predict(X_val_vectors_tfidf)
y_prob = svc_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]

print('SVC + tf-idf')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
svctfidf_f1 = pd.DataFrame(f1_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

svctfidf_p = pd.DataFrame(precision_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

svctfidf_r = pd.DataFrame(recall_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

svc_tfidf_report = classification_report(y_val,y_predict, output_dict=True)
svctfidf = pd.DataFrame(svc_tfidf_report).transpose()

auc_list=[]

auc_list.append(roc_auc)
svctfidf_auc = pd.DataFrame(auc_list, columns=['auc'])

In [ ]:
print("Model F1 Score: ", f1_score(y_val,y_predict))
print("Model F1 mean, CI Lower, CI Upper, STD: ",f1_conf_int(y_val,y_predict))#gives f1 mean, ci lower, ci upper, std
print("Model Precision: ", precision_score(y_val, y_predict))
print("Model Precision mean, CI Lower, CI Upper, STD: ", precision_conf_int(y_val,y_predict))
print("Model Recall: ", recall_score(y_val, y_predict))
print("Model Recall mean, CI Lower, CI Upper, STD: ", recall_conf_int(y_val,y_predict))

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Linear SVC (tf-idf) ---- try

lsvc_tfidf = LinearSVC(multi_class='ovr', C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      penalty='l2', random_state=0, tol=1e-05, verbose=0)
lsvc_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = lsvc_tfidf.predict(X_val_vectors_tfidf)
y_prob = lsvc_tfidf._predict_proba_lr(X_val_vectors_tfidf)[:,1]

print('Linear SVC + tf-idf')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
lsvctfidf_f1 = pd.DataFrame(f1_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

lsvctfidf_p = pd.DataFrame(precision_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

lsvctfidf_r = pd.DataFrame(recall_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

lsvc_tfidf_report = classification_report(y_val,y_predict, output_dict=True)
lsvctfidf = pd.DataFrame(lsvc_tfidf_report).transpose()

auc_list=[]

auc_list.append(roc_auc)
lsvctfidf_auc = pd.DataFrame(auc_list, columns=['auc'])

In [ ]:
#FITTING THE CLASSIFICATION MODEL using SGD (tf-idf) ---- try

from sklearn.calibration import CalibratedClassifierCV

sgd_tfidf = SGDClassifier(loss='hinge',class_weight='balanced')
clf = sgd_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train_vectors_tfidf, y_train)

#Predict y value for test dataset
y_predict = model.predict(X_val_vectors_tfidf)
y_prob = model.predict_proba(X_val_vectors_tfidf)[:,1]

print('SGD + tf-idf')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
sgdtfidf_f1 = pd.DataFrame(f1_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

sgdtfidf_p = pd.DataFrame(precision_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

sgdtfidf_r = pd.DataFrame(recall_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

sgd_tfidf_report = classification_report(y_val,y_predict, output_dict=True)
sgdtfidf = pd.DataFrame(sgd_tfidf_report).transpose()

auc_list=[]

auc_list.append(roc_auc)
sgdtfidf_auc = pd.DataFrame(auc_list, columns=['auc'])

In [ ]:
#FITTING THE CLASSIFICATION MODEL using K neighbours (tf-idf) ---- try

kn_tfidf = KNeighborsClassifier()
kn = kn_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

calibrator = CalibratedClassifierCV(kn, cv='prefit')
model=calibrator.fit(X_train_vectors_tfidf, y_train)

#Predict y value for test dataset
y_predict = model.predict(X_val_vectors_tfidf)
y_prob = model.predict_proba(X_val_vectors_tfidf)[:,1]


print('KNN + tf-idf')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
knntfidf_f1 = pd.DataFrame(f1_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

knntfidf_p = pd.DataFrame(precision_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

knntfidf_r = pd.DataFrame(recall_conf_int(y_val, y_predict), index =list1,
                                              columns =['val'])

knn_tfidf_report = classification_report(y_val,y_predict, output_dict=True)
knntfidf = pd.DataFrame(knn_tfidf_report).transpose()

auc_list=[]

auc_list.append(roc_auc)
knntfidf_auc = pd.DataFrame(auc_list, columns=['auc'])

In [ ]:
#combine all classification reports
frames=[lrtfidf,nbtfidf, rftfidf, dttfidf, svctfidf, lsvctfidf, sgdtfidf, knntfidf]
result = pd.concat(frames, keys=["lr_tfidf", "nb_tfidf", "rf_tfidf","dt_tfidf","svc_tfidf","lsvc_tfidf","sgd_tfidf","knn_tfidf"])
result.head(10)

result.to_csv('./classification_reports.csv', index= True)

In [ ]:
auc_frames=[lrtfidf_auc,nbtfidf_auc, rftfidf_auc, dttfidf_auc, svctfidf_auc, lsvctfidf_auc, sgdtfidf_auc, knntfidf_auc]
auc_result = pd.concat(auc_frames, keys=["lr_tfidf", "nb_tfidf", "rf_tfidf","dt_tfidf","svc_tfidf","lsvc_tfidf","sgd_tfidf","knn_tfidf"])
auc_result.head(10)

auc_result.to_csv('./AUC_reports.csv', index= True)

In [ ]:
f1_frames=[lrtfidf_f1,nbtfidf_f1, rftfidf_f1, dttfidf_f1, svctfidf_f1, lsvctfidf_f1, sgdtfidf_f1, knntfidf_f1]
f1_result = pd.concat(f1_frames, keys=["lr_tfidf", "nb_tfidf", "rf_tfidf","dt_tfidf","svc_tfidf","lsvc_tfidf","sgd_tfidf","knn_tfidf"])
f1_result.head(10)

f1_result.to_csv('./f1_with_ci_reports.csv', index= True)


In [ ]:
p_frames=[lrtfidf_p,nbtfidf_p, rftfidf_p, dttfidf_p, svctfidf_p, lsvctfidf_p, sgdtfidf_p, knntfidf_p]
p_result = pd.concat(p_frames, keys=["lr_tfidf", "nb_tfidf", "rf_tfidf","dt_tfidf","svc_tfidf","lsvc_tfidf","sgd_tfidf","knn_tfidf"])
p_result.head(10)

p_result.to_csv('./p_with_ci_reports.csv', index= True)


In [ ]:
r_frames=[lrtfidf_r,nbtfidf_r, rftfidf_r, dttfidf_r, svctfidf_r, lsvctfidf_r, sgdtfidf_r, knntfidf_r]
r_result = pd.concat(r_frames, keys=["lr_tfidf", "nb_tfidf", "rf_tfidf","dt_tfidf","svc_tfidf","lsvc_tfidf","sgd_tfidf","knn_tfidf"])
r_result.head(10)

r_result.to_csv('./r_with_ci_reports.csv', index= True)


In [ ]:
!pwd

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression (W2v)
lr_w2v=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v.fit(X_train_vectors_w2v, y_train)  #model

#Predict y value for test dataset
y_predict = lr_w2v.predict(X_val_vectors_w2v)
y_prob = lr_w2v.predict_proba(X_val_vectors_w2v)[:,1]

print('Logistic regression + w2v')
print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

## Run non BERT classifier on test data

In [ ]:
testing_data.head()

In [ ]:
#Testing it on new dataset with the best model --- redo this with SVM classifier model
#df_test=pd.read_csv('test.csv')  #reading the data
df_test = testing_data
df_test['clean_text'] = df_test['context'].apply(lambda x: finalpreprocess(x)) #preprocess the data
X_test=df_test['clean_text']
X_vector=tfidf_vectorizer.transform(X_test) #converting X_test to vector
#y_predict = lr_tfidf.predict(X_vector)      #use the trained model on X_vector
#y_predict = svc_tfidf.predict(X_vector)
y_predict = rf_tfidf.predict(X_vector)
#y_prob = lr_tfidf.predict_proba(X_vector)[:,1]
#y_prob = svc_tfidf.predict_proba(X_vector)[:,1]
y_prob = rf_tfidf.predict_proba(X_vector)[:,1]
df_test['predict_prob']= y_prob
df_test['label']= y_predict
print(df_test.head())
#final=df_test[['id','label']].reset_index(drop=True)
df_test.to_csv(path+'rf_testing_label_pred.csv')

In [ ]:
df_test = pd.read_csv('pain_sentences_for_classifier_160k_to_174k.csv')
df_test.head()

In [ ]:
#Testing it on new dataset with the best model --- redo this with SVM classifier model
#df_test=pd.read_csv('test.csv')  #reading the data
#df_test = testing_data
df_test['clean_text_2'] = df_test['clean_text'].apply(lambda x: finalpreprocess(x)) #preprocess the data
X_test=df_test['clean_text_2']
X_vector=tfidf_vectorizer.transform(X_test) #converting X_test to vector
#y_predict = lr_tfidf.predict(X_vector)      #use the trained model on X_vector
#y_predict = svc_tfidf.predict(X_vector)
y_predict = rf_tfidf.predict(X_vector)
#y_prob = lr_tfidf.predict_proba(X_vector)[:,1]
#y_prob = svc_tfidf.predict_proba(X_vector)[:,1]
y_prob = rf_tfidf.predict_proba(X_vector)[:,1]
df_test['predict_prob']= y_prob
df_test['label']= y_predict
print(df_test.head())
#final=df_test[['id','label']].reset_index(drop=True)
df_test.to_csv(path+'rf_tfidf_160k_to_174k.csv')

# Other links to look at

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.PrecisionRecallDisplay.html#sklearn.metrics.PrecisionRecallDisplay.from_estimator

https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py

https://machinelearningmastery.com/report-classifier-performance-confidence-intervals/

https://stackoverflow.com/questions/66814430/seaborn-boxplot-with-confidence-intervals-for-the-mean?rq=1

https://stackoverflow.com/questions/50062201/boxplot-in-pandas-with-confidence-intervals-and-bootstrap-returns-exception-re

https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html#matplotlib.pyplot.boxplot  -- bootstrap

https://machinelearningmastery.com/report-classifier-performance-confidence-intervals/ --formulas